## Analyzing and fitting interferometric visibility datasets of circumstellar rings

Bringing your data from archive to paper-ready plots requires several steps, which we will follow and execute in this Notebook. Built in Python 3 and with CASA 5.4.0, but may handle other versions.

Needs GALARIO package:
https://github.com/mtazzari/galario
to be installed, e.g. with conda with a command like: 'conda install -c conda-forge galario'. This code was tested with version 1.2

## Step 0: Create list with all requested inputs

Here you need to decide if you are doing imaging and/or fitting and/or 
input some parameters that are needed for the imaging and/or fitting and/or postprocessing.


In [2]:
import numpy as np
import os, subprocess
import pickle

#Decide which you want to do, imaging and/or fitting and/or postprocessing
imaging=True
fit=True
postproc=True

radmcgalapath='/d1/boudica1/lmatra/radmc-gala'
casapath='/d1/boudica1/casa-release-5.4.0-68.el6/bin'
sourcetag='GJ14'
workingdir='/d1/boudica1/lmatra'
vis=['/sma/SMAusers/lmatra/REASONS_ALMA/cycle5/GJ14/calibratedms/GJ14_calibratedvis_cont.ms']
nvis=len(vis)
if not os.path.exists(workingdir+'/'+sourcetag):
    os.mkdir(workingdir+'/'+sourcetag)
    print('Creating directory for object '+sourcetag+' at '+workingdir)
else:
    print('Directory for object '+sourcetag+' already exists at '+workingdir)

if imaging:
    #Imaging parameters
    mosaic=False
    if mosaic:
        mosaic=True
        phasecenter='J2000 22h57m39.449700 -29d37m22.68681'
    else:
        phasecenter=''
    weighting='natural'
    if weighting=='briggs':
        robust=0.5
    else:
        robust=''
    uvtaper=['']
    interactive=True

if fit:
    #Parameters for fit
    Lstar=0.111  #Solar luminosities
    dist=14.7  #pc
    imsize=24.0     #arcsec, used for radiative transfer. >>2x belt outer radius, but not too large or it will slow down computation.
    #imsize above is also the size of the grid over which the model is setup in RADMC. No disk will be put beyond this.

    #Add disk parameters
    fluxdensity=1.8e-3 #Jy
    rmid=99.0/dist # Radial peak location of Gaussian surface density, arcsec
    sigma=32.0/2.35/dist #standard deviation of radial Gaussian surface density
    useh=True
    if useh:
        h=0.05 #aspect ratio of belt, constant with radius, whose vertical density structure at radius r is a Gaussian with width hr.
    else:
        h=0.03
    incl=65.0 #inclination, degrees from face-on
    posang=5.5 #position angle, East of North.

    #Add star if wanted/needed
    star=True
    if star:
        fstar=4e-5 #Jy
    
    #Add extra parameters independently for each of the visibility datasets.
    if nvis>=1:
        dRA=[-0.07,0.1,0.1] #RA offset of star+disk from phase center of observations
        dDec=[0.14,-0.1,-0.1] #Dec offset of star+disk from phase center of observations
        wtfact=[0.289,0.289,1e-4] #factors by which incorrect weights should be multiplied by
    else:
        dRA=[-0.07] #RA offset of star+disk from phase center of observations
        dDec=[0.14] #Dec offset of star+disk from phase center of observations
        wtfact=[0.289] #factors by which incorrect weights should be multiplied by

    #Add galaxies if needed
    ngal=0
    if ngal>=1:
        resolved=[True, False, False] # if definitely resolved, use 2D Gaussian as galaxy model (6 free parameters), and set resolved=True for that galaxy. 
    #Otherwise, use point source (3 free parameters) by setting resolved=False.
        fbkg=[285e-6,500e-6,500e-6] #Flux (Jy)
        dRAbkg=[2.97,2.97,2.97] # RA offset (")
        dDecbkg=[1.81,1.81,1.81] # Dec offset (")
        sigmagal=[0.3,0,0] # sigma (")
        PAgal=[28.0,0,0] # PA (deg)
        incgal=[49.0,0,0] # inc (deg)
    else: 
        resolved=None
        fbkg=None
        dRAbkg=None
        dDecbkg=None
        sigmagal=None
        PAgal=None
        incgal=None


Directory for object GJ14 already exists at /d1/boudica1/lmatra


## Step 1: Create directory structure

In [3]:
print('Creating directory structure')
os.chdir(workingdir+'/'+sourcetag)

#Imaging
if imaging:
    print('Will be carrying out imaging')
    for i in ['calibratedms', 'imaging']:
        if not os.path.exists(workingdir+'/'+sourcetag+'/'+i):
            os.mkdir(workingdir+'/'+sourcetag+'/'+i)
    !cp -r {radmcgalapath}/utils/mstonumpyortxt_multiple.py {workingdir}/{sourcetag}/calibratedms/.
    for i in np.arange(len(vis)):
        if not os.path.exists('calibratedms/'+vis[i].rsplit('/',1)[1]):
            !cp -r {vis[i]} {workingdir}/{sourcetag}/calibratedms/.
        vis[i]=vis[i].rsplit('/',1)[1]
        #print(vis[i])
    !cp -r {radmcgalapath}/utils/imagingscript_multiple.py {workingdir}/{sourcetag}/imaging/.  
    #Save imaging parameters
    pickle.dump([sourcetag,workingdir,vis,nvis,mosaic,phasecenter,weighting,robust,uvtaper,interactive], open(workingdir+'/'+sourcetag+'/imaging/imagepars.npy', 'wb'), protocol=2)

    
#Fitting
if fit:
    print('Will be carrying out visibility fit')
    if not os.path.exists(workingdir+'/'+sourcetag+'/'+'imaging'):
        sys.exit('Carry out imaging first: need .pb primary beam image')
    for i in ['uvfit']:
        if not os.path.exists(workingdir+'/'+sourcetag+'/'+i):
            os.mkdir(workingdir+'/'+sourcetag+'/'+i)
    !cp -r {radmcgalapath}/utils/runfit.py {workingdir}/{sourcetag}/uvfit/. 
    !cp -r {radmcgalapath}/utils/problem_setup_cont_gauss.py {workingdir}/{sourcetag}/uvfit/.  
    !cp -r {radmcgalapath}/utils/dustkappa_10445.micr.inp {workingdir}/{sourcetag}/uvfit/.  
    #Save fit parameters
    pickle.dump([Lstar,dist,imsize, fluxdensity,rmid,sigma,useh,h,incl,posang,star,fstar,nvis,dRA,dDec,wtfact
             ,ngal,resolved,fbkg,dRAbkg,dDecbkg,sigmagal,PAgal,incgal], open(workingdir+'/'+sourcetag+'/uvfit/fitpars.npy', 'wb'), protocol=2)

#Postprocessing
if postproc:
    if not (imaging or fit):
        print ('Will be carrying out postprocessing ONLY')
    else:
        print('Will be carrying out postprocessing')
    if not os.path.exists(workingdir+'/'+sourcetag+'/'+'uvfit'):
        sys.exit('Carry out fit before POSTprocessing!')
    for i in ['analysis', 'plots', 'uvfit/evaluation']:
        if not os.path.exists(workingdir+'/'+sourcetag+'/'+i):
            os.mkdir(workingdir+'/'+sourcetag+'/'+i)
    !cp -r {radmcgalapath}/utils/evaluatemodel_radmc3d.py {workingdir}/{sourcetag}/uvfit/. 
    !cp -r {radmcgalapath}/utils/uvresidualtoms.py {workingdir}/{sourcetag}/uvfit/. 
    !cp -r {radmcgalapath}/utils/makeuvdeprojplot_simple_multiple.py {workingdir}/{sourcetag}/uvfit/.
    !cp -r {radmcgalapath}/utils/plotimage.py {workingdir}/{sourcetag}/analysis/.  
    !cp -r {radmcgalapath}/utils/imagecombo.py {workingdir}/{sourcetag}/analysis/. 
   


Creating directory structure
Will be carrying out imaging
GJ14_calibratedvis_cont.ms
Will be carrying out visibility fit
cp: cannot stat `/d1/boudica1/lmatra/radmc-gala/utils/runfit.py': No such file or directory
cp: cannot stat `/d1/boudica1/lmatra/radmc-gala/utils/problem_setup_cont_gauss.py': No such file or directory
cp: cannot stat `/d1/boudica1/lmatra/radmc-gala/utils/dustkappa_10445.micr.inp': No such file or directory
Will be carrying out postprocessing
cp: cannot stat `/d1/boudica1/lmatra/radmc-gala/utils/evaluatemodel_radmc3d.py': No such file or directory
cp: cannot stat `/d1/boudica1/lmatra/radmc-gala/utils/uvresidualtoms.py': No such file or directory
cp: cannot stat `/d1/boudica1/lmatra/radmc-gala/utils/makeuvdeprojplot_simple_multiple.py': No such file or directory
cp: cannot stat `/d1/boudica1/lmatra/radmc-gala/utils/plotimage.py': No such file or directory
cp: cannot stat `/d1/boudica1/lmatra/radmc-gala/utils/imagecombo.py': No such file or directory


## Step 2: Carry out imaging via imagingscript_multiple CASA script
First convert visibilities in CASA MS format to a python save file

In [4]:
os.chdir('calibratedms')
!{casapath}/casa -c mstonumpyortxt_multiple.py


The start-up time of CASA may vary
depending on whether the shared libraries
are cached or not.

]0;IPython: GJ14/calibratedmsIPython 5.1.0 -- An enhanced Interactive Python.

CASA 5.4.0-68   -- Common Astronomy Software Applications

--> CrashReporter initialized.
Found data with 225826 uv points per channel
with 1 channels per SPW and 2 polarizations,
8 SPWs and Channel 0 frequency of 1st SPW of 243.116401246 GHz
corresponding to 1.23311302102 mm
Datasets has baselines between 11.6500031801 and 371.63538144 m


Read in visibilities into Python, and use Galario to figure out ideal cell size and image size.
Typically half the suggested image size is OK.

In [6]:
from galario.double import get_image_size
u=[[] for x in vis]
v=[[] for x in vis]
Re=[[] for x in vis]
Im=[[] for x in vis]
w=[[] for x in vis]
nxy=[[] for x in vis]
dxy=[[] for x in vis]
for i in np.arange(nvis):
    u[i], v[i], Re[i], Im[i], w[i] = np.load(vis[i][:-3]+'.npy')
    nxy[i], dxy[i] = get_image_size(u, v)
    nxy[i]/=2
    print('Pixel size (arcsec) and number of pixels required for dataset '+vis[i]+':')
    print(dxy[i]*180.0/np.pi*3600, nxy[i])    

#Save pixel sizes and number of pixels, for imaging and fitting
pickle.dump([dxy, list(map(int, nxy))], open(workingdir+'/'+sourcetag+'/calibratedms/pixinfo.npy', 'wb'), protocol=2)


Pixel size (arcsec) and number of pixels required for dataset GJ14_calibratedvis_cont.ms:
0.06843568580635173 512.0


Then run the CLEANing using CASA's tclean.

Run this within CASA locally. There is no way around this if you want to use the interactive cleaning mode!

In [7]:
os.chdir('../imaging')
#THE COMMAND BELOW to be run on local computer for CASA to bring up interactive prompt
execfile('imagingscript_multiple.py')

NameError: name 'execfile' is not defined

## Step 3: Run visibility fit by forward-modeling with RADMC-3D (ray tracing) and GALARIO (for FFT) through emcee package

In [11]:
os.chdir('../uvfit')


[array([33515.66666037, 27327.65606168, 83329.27393947, ...,
         8950.090162  ,  3090.27655607, -5859.83833311])]